In [ ]:
def processes_data():
    pass

In [ ]:
def model():
    pass

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/CoilData.csv')
df.head()

In [ ]:
#print(df.dtypes)
df.columns.to_list

In [ ]:
columns_to_drop = ['Constriction_width_x', 'Is_constriction_x',
       'Constriction_width_y', 'Is_constriction_y', 'Constriction_width',
       'Is_constriction_x.1', 'Is_constriction_y.1']  
df = df.drop(columns=columns_to_drop) 

df[df.isnull().any(axis=1)].count()

df.shape
df['Is_constriction'].value_counts()

In [ ]:
pd.crosstab(df['analyse'], df['Is_constriction'])
df.describe()

In [ ]:
# to speed up the calculations 
# pd.Categorical(df['analyse'])

In [ ]:
#optional
#df_no_a = df.drop(columns='analyse') 



num_attribs = ['furnace Number', 'Hardness_1', 'Hardness_2', 'Width', 'Temperature before finishing mill','Temperature after finishing mill', 'Thickness', 'Thickness profile', 'C', 'Mn', 'Si', 'Nb', 'P', 'S', 'Al', 'M_Al', 'B', 'N', 'Ti', 'Cr', 'V', 'Mo']
cat_attribs = ["analyse"]
regressor = RandomForestRegressor(n_estimators=5, warm_start=True)
onehot = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()
#classifier = RandomForestRegressor(n_estimators=5, warm_start=True)
pipeline = Pipeline(steps=[
    ('columtransformer', ColumnTransformer([
        ('std_scaler', scaler, num_attribs),
        ('one_hot_encoder', onehot, cat_attribs)
    ])),
    ('classifier', regressor)
])
model = pipeline.fit(X_train, y_train) 

In [ ]:
FEATURES = list(df.columns[1:-1])

FEATURES

result_dict = {}

In [ ]:
def summarize_classification(y_test, y_pred):
    
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)

    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy': acc, 
            'precision': prec,
            'recall':recall, 
            'accuracy_count':num_acc}

In [ ]:
def build_model(classifier_fn,                
                name_of_y_col, 
                names_of_x_cols, 
                dataset, 
                test_frac=0.2):
    
    X = dataset[names_of_x_cols]
    Y = dataset[name_of_y_col]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
       
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary, 
            'test': test_summary,
            'confusion_matrix': model_crosstab}

In [ ]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)

        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
       
        print()


In [ ]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [ ]:
result_dict['constriction ~ logistic'] = build_model(logistic_fn,
                                              'Is_constriction',
                                               FEATURES,
                                               df)

compare_results()